In [22]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup


In [16]:
def scrape_ken_pom():
    
    url="https://kenpom.com/index.php"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    #find the table and header tags
    table = soup.find_all('table', {'id':'ratings-table'})[0]
    headers = table.find('thead').find('tr', {'class':'thead2'})
    #store the header tags titles into a list
    cols = [th.text for th in headers.find_all('th')]

    #initialize the dataframe
    
    kenpom_df = pd.DataFrame(columns=cols)

    #get the data elements from the table
    body = table.find_all('tbody')[0]
    rows = body.find_all('tr')

    #scrape the data from each row
    for r in rows:
        data = r.find_all('td')
        info = []

        for d in data:
            try:
                if(d["class"] != ['td-right']):
                    info.append(d.text)
            except:
                info.append(d.text)

        #if the list isnt empty
        if(info != []):
            #create a series with the list and add it to the dataframe
            info_df = pd.Series(info, index=cols)
            kenpom_df = kenpom_df.append(info_df, ignore_index=True)

    #clean and convert columns
    kenpom_df["AdjO"] = pd.to_numeric(kenpom_df["AdjO"])
    kenpom_df["AdjD"] = pd.to_numeric(kenpom_df["AdjD"])
    kenpom_df["AdjT"] = pd.to_numeric(kenpom_df["AdjT"])

    return kenpom_df

In [23]:
kenpom_df = scrape_ken_pom()

In [6]:
from sportsipy.ncaab.teams import Teams
teams = Teams()

In [28]:
sportsipy_team_names = [t.name for t in teams]
kenpom_names = list(kenpom_df.Team.values)

In [34]:
matches = [name for name in sportsipy_team_names if name in kenpom_names]

In [37]:
for name in sportsipy_team_names:
    if(name not in kenpom_names):
        print(name)

Alabama-Birmingham
Alabama State
Albany (NY)
Alcorn State
Appalachian State
Arizona State
Arkansas-Pine Bluff
Arkansas State
Ball State
Boise State
Bowling Green State
Brigham Young
Cal State Bakersfield
Cal State Fullerton
Cal State Northridge
California Baptist
UC-Davis
UC-Irvine
UC-Riverside
UC-San Diego
UC-Santa Barbara
University of California
Central Connecticut State
Central Florida
Chicago State
Citadel
Cleveland State
College of Charleston
Colorado State
Coppin State
Delaware State
Detroit Mercy
Dixie State
East Tennessee State
Florida International
Florida State
Fresno State
Gardner-Webb
Georgia State
Grambling
Idaho State
Illinois-Chicago
Illinois State
Indiana State
Iowa State
Purdue-Fort Wayne
Jackson State
Jacksonville State
Kansas State
Kennesaw State
Kent State
Cal State Long Beach
Long Island University
Louisiana-Monroe
Louisiana State
Loyola (IL)
Loyola (MD)
Maryland-Baltimore County
Maryland-Eastern Shore
Massachusetts-Lowell
McNeese State
Miami (FL)
Miami (OH)
Michi